In [1]:
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('.\Graph-WaveNet')
from model import *

# Model Class documentation

```python
def __init__(self, device, num_nodes, dropout=0.3, supports=None, do_graph_conv=True,
                 addaptadj=True, aptinit=None, in_dim=2, out_dim=12,
                 residual_channels=32, dilation_channels=32, cat_feat_gc=False,
                 skip_channels=256, end_channels=512, kernel_size=2, blocks=4, layers=2,
                 apt_size=

    @classmethod
    def from_args(cls, args, device, supports, aptinit, **kwargs):
        defaults = dict(dropout=args.dropout, supports=supports,
                        do_graph_conv=args.do_graph_conv, addaptadj=args.addaptadj, aptinit=aptinit,
                        in_dim=args.in_dim, apt_size=args.apt_size, out_dim=args.seq_length,
                        residual_channels=args.nhid, dilation_channels=args.nhid,
                        skip_channels=args.nhid * 8, end_channels=args.nhid * 16,
                        cat_feat_gc=args.cat_feat_gc)
        defaults.update(**kwargs)
        model = cls(device, args.num_nodes, **defaults)
        ret``` model
```10):

Here is a series of arguments that need explanation or clarification:

*  ```device```: torch device, i.e., the output of ```torch.device('cpu')```.
* ```num_nodes ```: node number.
* ```in_dim ```:
* ```out_dim ```:
* ```residual_channels```:
* ```dilation_channels```:
* ```skip_channels```:
* ```end_channels```:
* ```blocks```:
* ```layers```:

In [3]:
DEFAULT_DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
GWNet = gwnet(DEFAULT_DEVICE,20)

## Input documentation

```python
def forward(self, x):
        # Input shape is (bs, features, n_nodes, n_timesteps)
        in_len = x.size(3)
        if in_len < self.receptive_field:
            x = nn.functional.pad(x, (self.receptive_field - in_len, 0, 0, 0))
        if self.cat_feat_gc:
            f1, f2 = x[:, [0]], x[:, 1:]
            x1 = self.start_conv(f1)
            x2 = F.leaky_relu(self.cat_feature_conv(f2))
            x = x1 + x2
        else:
            x = self.start_conv(x)
        skip = 0

> Input shape is (bs, features, n_nodes, n_timesteps)
* ```bs```: has to be batch size, in our case this has to be each sample
* ```features```: values?
* ```n_nodes```: has to be the value a each node??
* ```n_timesteps```: values at each time step

# Trainer Class documentation

```python
class Trainer():
    def __init__(self, model: GWNet, scaler, lrate, wdecay, clip=3, lr_decay_rate=.97)

    def train(self, input, real_val):
        self.model.train()
        self.optimizer.zero_grad()
        input = nn.functional.pad(input,(1,0,0,0))
        output = self.model(input).transpose(1,3)  # now, output = [batch_size,1,num_nodes, seq_length]
        predict = self.scaler.inverse_transform(output):

# Training Documentation

```python
def main(args, **model_kwargs):
    device = torch.device(args.device)
    data = util.load_dataset(args.data, args.batch_size, args.batch_size, args.batch_size, n_obs=args.n_obs, fill_zeroes=args.fill_zeroes)
    scaler = data['scaler']
    aptinit, supports = util.make_graph_inputs(args, device)

    model = GWNet.from_args(args, device, supports, aptinit, **model_k


    for _ in mb:
        train_loss, train_mape, train_rmse = [], [], []
        data['train_loader'].shuffle()
        for iter, (x, y) in enumerate(data['train_loader'].get_iterator()):
            trainx = torch.Tensor(x).to(device).transpose(1, 3)
            trainy = torch.Tensor(y).to(device).transpose(1, 3)
            yspeed = trainy[:, 0, :, :]
            if yspeed.max() == 0: continue
            mae, mape, rmse = engine.train(trainx, yspeed)
            train_loss.append(mae)
            train_mape.append(mape)
            train_rmse.append(rmse)wargs)

## Data Loader documentation

```python
class DataLoader(object):
    def __init__(self, xs, ys, batch_size, pad_with_last_sample=True):
        """
        :param xs:
        :param ys:
        :param batch_size:
        :param pad_with_last_sample: pad with the last sample to make number of samples divisible to batch_size.
        """
        self.batch_size = batch_size
        self.current_ind = 0
        if pad_with_last_sample:
            num_padding = (batch_size - (len(xs) % batch_size)) % batch_size
            x_padding = np.repeat(xs[-1:], num_padding, axis=0)
            y_padding = np.repeat(ys[-1:], num_padding, axis=0)
            xs = np.concatenate([xs, x_padding], axis=0)
            ys = np.concatenate([ys, y_padding], axis=0)
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
 

def load_dataset(dataset_dir, batch_size, valid_batch_size=None, test_batch_size=None, n_obs=None, fill_zeroes=True):
    data = {}
    for category in ['train', 'val', 'test']:
        cat_data = np.load(os.path.join(dataset_dir, category + '.npz'))
        data['x_' + category] = cat_data['x']
        data['y_' + category] = cat_data['y']
        if n_obs is not None:
            data['x_' + category] = data['x_' + category][:n_obs]
            data['y_' + category] = data['y_' + category][:n_obs]
    scaler = StandardScaler(mean=data['x_train'][..., 0].mean(), std=data['x_train'][..., 0].std(), fill_zeroes=fill_zeroes)
    # Data format
    for category in ['train', 'val', 'test']:
        data['x_' + category][..., 0] = scaler.transform(data['x_' + category][..., 0])
    data['train_loader'] = DataLoader(data['x_train'], data['y_train'], batch_size)
    data['val_loader'] = DataLoader(data['x_val'], data['y_val'], valid_batch_size)
    data['test_loader'] = DataLoader(data['x_test'], data['y_test'], test_batch_size)
    data['scaler'] = scaler
    return data       self.ys = ys

# Data Format

In [9]:
import h5py
 
#Open the H5 file in read mode
with h5py.File('./data/metr-la.h5', 'r') as file:
    print("Keys: %s" % file.keys())
    a_group_key = list(file.keys())[0]
    print(list(file.keys()))
     
    # Getting the data
    data = list(file[a_group_key])
    print(data)
    print(file['df'])

    group = file['/df']
    
    # Iterate through the members of the group
    for member in group:
        # Read data from each member
        data = group[member][()]
        print(f"Data from {member}:")
        print(data)

Keys: <KeysViewHDF5 ['df']>
['df']
['axis0', 'axis1', 'block0_items', 'block0_values']
<HDF5 group "/df" (4 members)>
Data from axis0:
[b'773869' b'767541' b'767542' b'717447' b'717446' b'717445' b'773062'
 b'767620' b'737529' b'717816' b'765604' b'767471' b'716339' b'773906'
 b'765273' b'716331' b'771667' b'716337' b'769953' b'769402' b'769403'
 b'769819' b'769405' b'716941' b'717578' b'716960' b'717804' b'767572'
 b'767573' b'773012' b'773013' b'764424' b'769388' b'716328' b'717819'
 b'769941' b'760987' b'718204' b'718045' b'769418' b'768066' b'772140'
 b'773927' b'760024' b'774012' b'774011' b'767609' b'769359' b'760650'
 b'716956' b'769831' b'761604' b'717495' b'716554' b'773953' b'767470'
 b'716955' b'764949' b'773954' b'767366' b'769444' b'773939' b'774067'
 b'769443' b'767750' b'767751' b'767610' b'773880' b'764766' b'717497'
 b'717490' b'717491' b'717492' b'717493' b'765176' b'717498' b'717499'
 b'765171' b'718064' b'718066' b'765164' b'769431' b'769430' b'717610'
 b'767053' b'

In [12]:
import pandas as pd
pd.read_hdf('/data/metr-la.h5')

FileNotFoundError: File /data/metr-la.h5 does not exist